In [2]:
"""
PART 1: Initialization and Setup
"""

import openai
import os
import json
import re
from datetime import datetime
from typing import Dict, List, Optional
import gradio as gr
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class HealthChatbot:
    """
    A health-focused chatbot with safety features and prompt engineering
    """
    
    def __init__(self, api_key: str = None):
        """
        Initialize the chatbot with API key and safety configurations
        """
        # Initialize OpenAI client
        self.api_key = api_key or os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OpenAI API key is required. Set it in .env file or pass directly.")
        
        openai.api_key = self.api_key
        self.client = openai.OpenAI(api_key=self.api_key)
        
        # Define safety keywords that should trigger warnings
        self.emergency_keywords = [
            'emergency', 'heart attack', 'stroke', 'bleeding', 'unconscious',
            'difficulty breathing', 'chest pain', 'severe pain', 'suicide', 
            'overdose', 'cannot breathe', 'choking'
        ]
        
        self.medication_keywords = [
            'dosage', 'dose', 'how much to take', 'how often', 'prescription',
            'medicine', 'medication', 'drug', 'tablet', 'pill'
        ]
        
        # Conversation history
        self.conversation_history = []
        
        print("✓ Health Chatbot initialized successfully!")
        print("✓ Safety filters are active")
        print("✓ Ready to answer health-related questions")

In [4]:
"""
PART 2: Safety Check Functions
"""

def check_emergency(self, query: str) -> bool:
        """
        Check if query contains emergency keywords
        """
        query_lower = query.lower()
        for keyword in self.emergency_keywords:
            if keyword in query_lower:
                return True
        return False
    
def check_medication_query(self, query: str) -> bool:
        """
        Check if query is about medication dosage
        """
        query_lower = query.lower()
        for keyword in self.medication_keywords:
            if keyword in query_lower:
                return True
        return False
    
def get_emergency_response(self) -> str:
        """
        Return emergency response message
        """
        return (
            "⚠️ **IMPORTANT SAFETY NOTICE** ⚠️\n\n"
            "Based on your query, it sounds like you might be experiencing a medical emergency. "
            "Please follow these steps immediately:\n\n"
            "1️⃣ **Call Emergency Services (911 or your local emergency number)**\n"
            "2️⃣ Go to the nearest Emergency Room\n"
            "3️⃣ Do not wait for online advice\n\n"
            "This chatbot cannot provide emergency medical assistance. "
            "Please seek immediate professional medical help."
        )
    
def get_medication_disclaimer(self) -> str:
        """
        Return medication disclaimer
        """
        return (
            "\n\n⚠️ **MEDICATION SAFETY NOTICE** ⚠️\n\n"
            "Please remember:\n"
            "• Always consult a doctor or pharmacist before taking any medication\n"
            "• Follow the dosage instructions on the package or as prescribed\n"
            "• Never share prescription medications with others\n"
            "• Check for allergies and potential drug interactions\n"
            "• Keep all medications out of reach of children\n\n"
            "This information is for educational purposes only."
        )

In [5]:
"""
PART 3: Query Processing and LLM Integration
"""

def create_system_prompt(self) -> str:
        """
        Create the system prompt with medical assistant guidelines
        """
        return """You are a friendly and knowledgeable health information assistant. Your role is to:

1. **Provide General Health Information**: Answer questions about common health conditions, symptoms, and wellness tips
2. **Be Educational**: Explain medical terms in simple, easy-to-understand language
3. **Stay Within Scope**: Only provide general information, not specific medical advice
4. **Be Safe**: Always include appropriate disclaimers and encourage professional medical consultation
5. **Be Empathetic**: Show compassion and understanding for health concerns
6. **Be Clear**: Use bullet points and simple language for easy understanding

**Important Guidelines:**
- Never diagnose specific conditions
- Never prescribe or recommend specific treatments
- Always encourage consulting healthcare professionals for personal medical advice
- For emergency situations, advise seeking immediate medical help
- Provide general wellness tips and preventive care information
- Explain possible causes of symptoms in general terms
- Discuss common treatments and their general purposes
- Share reliable health resources and information sources

**Tone:**
- Warm and friendly like a caring health educator
- Professional but not intimidating
- Clear and easy to understand
- Reassuring without being dismissive of concerns

Remember: You are here to educate and inform, not to replace professional medical advice.
"""
    
def process_query(self, user_query: str) -> str:
        """
        Process user query and generate response with safety checks
        """
        # Check for emergency
        if self.check_emergency(user_query):
            return self.get_emergency_response()
        
        # Add to conversation history
        self.conversation_history.append({
            "role": "user",
            "content": user_query,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })
        
        try:
            # Create messages for the API
            messages = [
                {"role": "system", "content": self.create_system_prompt()},
                {"role": "user", "content": f"Please answer this health question in a helpful, educational way: {user_query}"}
            ]
            
            # Add conversation context if available
            if len(self.conversation_history) > 1:
                recent_history = self.conversation_history[-3:]  # Last 3 exchanges
                for exchange in recent_history:
                    messages.append({"role": exchange["role"], "content": exchange["content"]})
            
            # Call OpenAI API
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=500,
                temperature=0.7,
                top_p=0.9
            )
            
            # Extract response
            bot_response = response.choices[0].message.content
            
            # Add medication disclaimer if needed
            if self.check_medication_query(user_query):
                bot_response += self.get_medication_disclaimer()
            
            # Add general disclaimer
            general_disclaimer = (
                "\n\n---\n*Disclaimer: This information is for educational purposes only. "
                "Always consult with a qualified healthcare professional for medical advice, "
                "diagnosis, or treatment.*"
            )
            bot_response += general_disclaimer
            
            # Store response in history
            self.conversation_history.append({
                "role": "assistant",
                "content": bot_response,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })
            
            return bot_response
            
        except Exception as e:
            error_message = f"❌ Error processing your query: {str(e)}"
            print(error_message)
            return "I'm having trouble processing your request right now. Please try again later."
    
def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []
        return "Conversation history cleared!"

In [ ]:
"""
PART 4: Gradio Web Interface
"""

def create_interface():
    """
    Create and configure the Gradio web interface
    """
    # Initialize chatbot
    chatbot = HealthChatbot()
    
    def respond(message, history):
        """Handle user messages and return response"""
        if not message.strip():
            return "Please enter a question."
        
        # Process the query
        response = chatbot.process_query(message)
        return response
    
    # Create Gradio interface
    with gr.Blocks(title="🏥 General Health Query Chatbot", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🏥 General Health Query Chatbot
        
        Welcome! I'm here to provide general health information and answer your wellness questions.
        
        **Please Note:** This chatbot provides educational information only and is not a substitute for professional medical advice.
        
        ### Example Questions:
        - What causes a sore throat?
        - How can I improve my sleep quality?
        - Is paracetamol safe for children?
        - What are common cold symptoms?
        - Tips for maintaining a healthy diet
        """)
        
        with gr.Row():
            with gr.Column(scale=4):
                chatbot_interface = gr.Chatbot(
                    label="Conversation",
                    height=400
                )
                msg = gr.Textbox(
                    label="Type your health question here",
                    placeholder="e.g., What causes a sore throat?",
                    lines=2
                )
                
                with gr.Row():
                    submit_btn = gr.Button("Send", variant="primary")
                    clear_btn = gr.Button("Clear Conversation")
        
        # Define interactions
        def user(user_message, history):
            return "", history + [[user_message, None]]
        
        def bot(history):
            user_message = history[-1][0]
            bot_response = respond(user_message, history[:-1])
            history[-1][1] = bot_response
            return history
        
        msg.submit(user, [msg, chatbot_interface], [msg, chatbot_interface], queue=False).then(
            bot, chatbot_interface, chatbot_interface
        )
        
        submit_btn.click(user, [msg, chatbot_interface], [msg, chatbot_interface], queue=False).then(
            bot, chatbot_interface, chatbot_interface
        )
        
        clear_btn.click(lambda: None, None, chatbot_interface, queue=False)
        
        # Add safety information
        with gr.Accordion("⚠️ Safety Information & Disclaimer", open=False):
            gr.Markdown("""
            ### Important Safety Guidelines:
            
            1. **Emergency Situations**: If you're experiencing a medical emergency, call emergency services immediately.
            
            2. **Professional Consultation**: Always consult with healthcare professionals for personal medical advice.
            
            3. **Information Purpose**: This chatbot provides general health information for educational purposes only.
            
            4. **No Diagnosis**: The chatbot cannot diagnose conditions or prescribe treatments.
            
            5. **Medication Safety**: Never start or stop medications without consulting a doctor.
            
            ### When to See a Doctor:
            - Severe or persistent symptoms
            - Unexplained weight loss
            - High fever
            - Difficulty breathing
            - Chest pain
            - Severe headaches
            - Changes in vision
            - Any concerning symptoms
            """)
    
    return demo